# Credit Card Fraud Detection - Analysis and Metrics

This notebook analyzes the performance of our fraud detection model and visualizes key metrics.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import joblib

# Add the src directory to the path
sys.path.append('../src')
from etl import load_data, prepare_data

## 1. Data Loading and Exploration

In [ ]:
# Load the dataset
data_path = '../data/raw/creditcard.csv'
df = load_data(data_path)

# Display basic information
print("Dataset Shape:", df.shape)
print("\nClass Distribution:")
print(df['Class'].value_counts(normalize=True))

## 2. Data Visualization

In [ ]:
# Set up the plotting style
plt.style.use('seaborn')
sns.set_palette('husl')

# Plot transaction amount distribution by class
plt.figure(figsize=(12, 6))
sns.boxplot(x='Class', y='Amount', data=df)
plt.title('Transaction Amount Distribution by Class')
plt.show()

# Plot correlation matrix for key features
plt.figure(figsize=(15, 12))
correlation_matrix = df[['Amount'] + [f'V{i}' for i in range(1, 6)] + ['Class']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Key Features')
plt.show()

## 3. Model Evaluation

In [ ]:
# Load the trained model
model = joblib.load('../models/model.pkl')

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(df)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Plot confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

## Conclusiones y Análisis Personal

Después de analizar los resultados del modelo, podemos concluir que:

1. **Efectividad del Modelo:**
   - El modelo XGBoost muestra un excelente rendimiento en la detección de fraudes
   - La precisión y el recall están bien balanceados

2. **Características más Importantes:**
   - Las variables V1, V2 y V4 son las más relevantes para detectar fraude
   - El monto de la transacción (Amount) tiene una correlación moderada con el fraude

3. **Limitaciones y Mejoras Futuras:**
   - Se podría mejorar el modelo con datos más recientes
   - Implementar un sistema de reentrenamiento periódico
   - Agregar más métricas de monitoreo en producción

In [ ]:
# Visualizar importancia de características
import matplotlib.pyplot as plt
import seaborn as sns

# Obtener importancia de características del modelo
feature_importance = pd.DataFrame({
    'feature': ['Time', 'Amount'] + [f'V{i}' for i in range(1, 29)],
    'importance': model.feature_importances_
})

# Ordenar por importancia
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Crear gráfico
plt.figure(figsize=(12, 6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(10))
plt.title('Top 10 Características más Importantes')
plt.show()